In [1]:
import re
import math
import numpy as np
import pandas as pd

# PREPROCESSING

In [2]:
from stop_word import stop_words
import nltk
from nltk.stem import PorterStemmer
import re
nltk.download('punkt') # Download data yang dibutuhkan untuk NLTK (jalankan sekali pada setiap instalasi)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df = pd.read_csv("./dataset/dataset.csv") # Memuat dataset

In [4]:
df

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1
...,...,...,...,...
4004,http://beforeitsnews.com/sports/2017/09/trends...,Trends to Watch,Trends to Watch\n% of readers think this story...,0
4005,http://beforeitsnews.com/u-s-politics/2017/10/...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,0
4006,https://www.activistpost.com/2017/09/ron-paul-...,"Ron Paul on Trump, Anarchism & the AltRight",NaN,0
4007,https://www.reuters.com/article/us-china-pharm...,China to accept overseas trial data in bid to ...,SHANGHAI (Reuters) - China said it plans to ac...,1


In [5]:
df.dropna(inplace=True) # Drop semua dataset bernilai NaN

In [6]:
df['Full'] = df['Headline'].loc[0:2000] + ' ' + df['Body'].loc[0:2000] # Membuat kolom baru full terdiri dari gabungan kolom headline dan body

In [7]:
df['Full']
df.dropna(inplace=True) # Drop semua dataset bernilai NaN

#### PREPROCESSING FUNCTION

In [8]:
def stemming(word):
    stemmer = PorterStemmer()
    stemmed_word = stemmer.stem(word)
    return stemmed_word

In [9]:
def stopword_removal_and_filtering(words):
    # Menghilangkan stopwords dan Case Folding
    filtered_words = [word for word in words if word.lower() not in stop_words] 

    # Filtering singkatan sederhana
    filtered_words = [re.sub(r'\.', '', word) for word in filtered_words]

    # Filtering angka
    filtered_words = [re.sub(r'\d', '', word) for word in filtered_words]
    
    # Filtering data redundan
    filtered_words = set(filtered_words)
    
    return filtered_words

In [55]:
def preprocessing(text):
    # Menghilangkan tanda baca
    text = re.sub(r'[^\w\s-]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Memisahkan teks menjadi kata-kata (Tokenizing)
    words = text.split()
    
    # Stop word removal dan stemming
    filtered_words = stopword_removal_and_filtering(words)

    # Stemming
    filtered_words = [stemming(i) for i in filtered_words]

    # Menghilangkan spasi ekstra
    processed_text = ' '.join(filtered_words)

    return processed_text


In [91]:
X_dataset_fullset = df['Full'].apply(preprocessing) # Proses preprocessing dataset

In [92]:
X_dataset_fullset

0       person  nixon-era unsuccess work flat-out spee...
1        fli loss owner screen drug benjamin stewart c...
2        absent mccain measur updat high disrupt role ...
3        reserv decades-long final develop wednesday e...
4       vega sing open la snl aldean live tom saturday...
                              ...                        
1996     red version warn stern public america crude h...
1997     tobi phenomenon hitchhik disappear great horr...
1998     transact vega varieti freebi comp perfect ste...
1999    return add fact prepar korea test reach carbon...
2000     racer lewi hamilton embrac prohibit start e-s...
Name: Full, Length: 1988, dtype: object

In [93]:
X_dataset_fullset = X_dataset_fullset.values.flatten() # Menggabungkan data menjadi 1 dimensi

In [94]:
X_dataset_fullset

array(['person  nixon-era unsuccess work flat-out speech day offici conserv - repeal tax quot member plead caucu crimin campaign chair york basi huge effort grow afghanistan collect vice-presid speak migrain theori challeng relat inabl spoke sign obamacar getti photo news speak republican white stir soft grumbl ongo choic diplomaci subject opposit charlottesvil unfett person iran coward brought ire interview imag commun state plan gift year thing presidenti administr reform target bite leader star continu fact qatar caption day report report hear chao deal re-elect major infrastructur adult stand healthcar calculu contradict good tennessean secretari refram face-palm insist remark donald lot sport kelli defend unpredict version endors list nationalist offhand akin essenti post tax peopl shake attempt liar daili rub real unrel controversi captur damn presid disput legion gener scrambl support invect testament legisl collaps silent staff hold torch-bear broader senat work primari off-scr

In [95]:
X_dataset_fullset = X_dataset_fullset.tolist() # Mengubah data menjadi list

In [96]:
X_dataset_fullset

['person  nixon-era unsuccess work flat-out speech day offici conserv - repeal tax quot member plead caucu crimin campaign chair york basi huge effort grow afghanistan collect vice-presid speak migrain theori challeng relat inabl spoke sign obamacar getti photo news speak republican white stir soft grumbl ongo choic diplomaci subject opposit charlottesvil unfett person iran coward brought ire interview imag commun state plan gift year thing presidenti administr reform target bite leader star continu fact qatar caption day report report hear chao deal re-elect major infrastructur adult stand healthcar calculu contradict good tennessean secretari refram face-palm insist remark donald lot sport kelli defend unpredict version endors list nationalist offhand akin essenti post tax peopl shake attempt liar daili rub real unrel controversi captur damn presid disput legion gener scrambl support invect testament legisl collaps silent staff hold torch-bear broader senat work primari off-script jo

In [97]:
X_string_fullset = ','.join(X_dataset_fullset) # Menggabung data menjadi sebuah string

In [98]:
X_feature_fullset = set(X_string_fullset.split(" ")) # Memisahkan string menjadi fitur - fitur
X_feature_fullset = list(X_feature_fullset)

In [99]:
X_feature_fullset = pd.Series(data=X_feature_fullset,name="Feature") # Mengubah list menjadi data series 

In [100]:
X_feature_fullset.dropna(inplace=True)

In [101]:
X_feature_fullset.drop(index=0,inplace=True)

In [102]:
X_feature_fullset.to_csv("./dataset/unique_words.csv")

## PERHITUNGAN BOBOT FITUR (TF-IDF)

In [112]:
import re
import math
import numpy as np
import pandas as pd

In [113]:
docs = X_dataset_fullset
processed_unique_words = pd.read_csv("./dataset/unique_words.csv",index_col=0)

In [123]:
processed_unique_words = processed_unique_words['Feature'].to_list()

In [114]:
tf = np.zeros((len(processed_unique_words), len(docs)))

### Menghitung Term Frequency

In [125]:
for i in range(len(docs)):
    for x in range(len(processed_unique_words)):
        count = docs[i].count(processed_unique_words[x])
        tf[x][i] = 0 if count == 0 else 1 + math.log(1 + count)

In [126]:
df_tf = pd.DataFrame(tf)

In [127]:
df_tf.to_csv('./dataset/tf.csv',index=False) # Menyimpan hasil perhitungan term frequency

## Data Preparation (START RUNNING DARI SINI)

### Load Feature

In [131]:
unique_word = pd.read_csv("./dataset/unique_words.csv",index_col=0)
unique_word = unique_word['Feature'].tolist()

In [132]:
tfdf = pd.read_csv("./dataset/tf.csv")
tfdf.index = unique_word
tfdf = tfdf.transpose()

In [133]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [134]:
X_train, X_test, y_train, y_test = train_test_split(tfdf, df['Label'], test_size=0.10, random_state=42) # Data Splitting

display(X_train,y_test)

,zuerlein,sprole,cole,"enrol,polic",nemati,aberdeen,grader,cupboard,potenti,retain,...,consid,azsport,gwen,up-fak,sharehold,fulmer,fast-forward,rapiscan,oppo,inflect
306,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.098612,0.0
921,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
353,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.098612,0.0
1294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.693147,0.0
1459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


1262    1
762     1
1390    1
70      0
1217    1
       ..
1197    1
1568    0
283     0
1203    0
1754    0
Name: Label, Length: 199, dtype: int64

In [135]:
# Normalisasi Data
norm_X_train = scaler.fit_transform(X_train)
norm_X_test = scaler.fit_transform(X_test)

## Create Model

In [136]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf

### Arsitektur 1

In [137]:
model = Sequential()

model.add(Dense(512, input_shape=(norm_X_train.shape[1],), activation='relu'))

# Hidden layers
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))  # Dropout layer for regularization

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout layer for regularization

# Output layer
model.add(Dense(1, activation='sigmoid'))  # Binary classification (fake or not)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(norm_X_train, y_train, epochs=100, batch_size=256, validation_split=0.1)

Epoch 1/100
7/7 [==============================] - 2s 139ms/step - loss: 0.6058 - accuracy: 0.6621 - val_loss: 0.3307 - val_accuracy: 0.8939
Epoch 2/100
7/7 [==============================] - 1s 112ms/step - loss: 0.2202 - accuracy: 0.9298 - val_loss: 0.1626 - val_accuracy: 0.9609
Epoch 3/100
7/7 [==============================] - 1s 111ms/step - loss: 0.0512 - accuracy: 0.9894 - val_loss: 0.2080 - val_accuracy: 0.9665
Epoch 4/100
7/7 [==============================] - 1s 112ms/step - loss: 0.0126 - accuracy: 0.9981 - val_loss: 0.2412 - val_accuracy: 0.9665
Epoch 5/100
7/7 [==============================] - 1s 117ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.2990 - val_accuracy: 0.9665
Epoch 6/100
7/7 [==============================] - 1s 115ms/step - loss: 0.0078 - accuracy: 0.9988 - val_loss: 0.4577 - val_accuracy: 0.9665
Epoch 7/100
7/7 [==============================] - 1s 113ms/step - loss: 0.0023 - accuracy: 0.9994 - val_loss: 0.1654 - val_accuracy: 0.9665
Epoch 8/100
7

In [138]:
loss, accuracy = model.evaluate(norm_X_test, y_test)

7/7 [==============================] - 0s 6ms/step - loss: 0.5284 - accuracy: 0.9598


### Arsitektur 2

In [139]:
model = Sequential()

# Input layer
model.add(Dense(512, input_shape=(norm_X_train.shape[1],), activation='relu'))
model.add(Dropout(0.6))

# Hidden layers
model.add(Dense(256, activation='relu'))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.6))

# Additional hidden layer
model.add(Dense(64, activation='relu'))


# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(norm_X_train, y_train, epochs=100, batch_size=256, validation_split=0.1)

Epoch 1/100
7/7 [==============================] - 2s 136ms/step - loss: 0.6559 - accuracy: 0.6224 - val_loss: 0.5087 - val_accuracy: 0.7318
Epoch 2/100
7/7 [==============================] - 1s 108ms/step - loss: 0.4204 - accuracy: 0.8335 - val_loss: 0.2349 - val_accuracy: 0.9385
Epoch 3/100
7/7 [==============================] - 1s 109ms/step - loss: 0.1739 - accuracy: 0.9366 - val_loss: 0.1608 - val_accuracy: 0.9665
Epoch 4/100
7/7 [==============================] - 1s 114ms/step - loss: 0.0734 - accuracy: 0.9783 - val_loss: 0.1499 - val_accuracy: 0.9721
Epoch 5/100
7/7 [==============================] - 1s 108ms/step - loss: 0.0290 - accuracy: 0.9950 - val_loss: 0.1348 - val_accuracy: 0.9665
Epoch 6/100
7/7 [==============================] - 1s 119ms/step - loss: 0.0179 - accuracy: 0.9969 - val_loss: 0.2226 - val_accuracy: 0.9721
Epoch 7/100
7/7 [==============================] - 1s 117ms/step - loss: 0.0117 - accuracy: 0.9963 - val_loss: 0.1188 - val_accuracy: 0.9777
Epoch 8/100
7

In [140]:
loss, accuracy = model.evaluate(norm_X_test, y_test)

7/7 [==============================] - 0s 5ms/step - loss: 0.2066 - accuracy: 0.9749


### Arsitektur 3

In [141]:
model = Sequential()

model.add(Dense(512, input_shape=(norm_X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))

# Hidden layers
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Additional hidden layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(norm_X_train, y_train, epochs=100, batch_size=256, validation_split=0.1)

Epoch 1/100
7/7 [==============================] - 2s 141ms/step - loss: 0.7083 - accuracy: 0.5137 - val_loss: 0.6813 - val_accuracy: 0.6872
Epoch 2/100
7/7 [==============================] - 1s 114ms/step - loss: 0.7140 - accuracy: 0.5366 - val_loss: 0.6805 - val_accuracy: 0.7821
Epoch 3/100
7/7 [==============================] - 1s 115ms/step - loss: 0.6989 - accuracy: 0.5596 - val_loss: 0.6706 - val_accuracy: 0.8101
Epoch 4/100
7/7 [==============================] - 1s 117ms/step - loss: 0.6830 - accuracy: 0.5839 - val_loss: 0.6572 - val_accuracy: 0.8212
Epoch 5/100
7/7 [==============================] - 1s 119ms/step - loss: 0.6452 - accuracy: 0.6273 - val_loss: 0.6215 - val_accuracy: 0.7933
Epoch 6/100
7/7 [==============================] - 1s 124ms/step - loss: 0.6258 - accuracy: 0.6733 - val_loss: 0.5746 - val_accuracy: 0.8324
Epoch 7/100
7/7 [==============================] - 1s 119ms/step - loss: 0.5740 - accuracy: 0.7360 - val_loss: 0.4980 - val_accuracy: 0.9050
Epoch 8/100
7

In [143]:
loss, accuracy = model.evaluate(norm_X_test, y_test)

7/7 [==============================] - 0s 14ms/step - loss: 0.3660 - accuracy: 0.9648


### ARSITEKTUR 4

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# model = Sequential()

# # Embedding layer
# model.add(Embedding(input_dim=2000, output_dim=128, input_length=norm_X_train.shape[1],))

# # LSTM layer
# model.add(LSTM(64, return_sequences=True))
# model.add(LSTM(32))

# # Dense layers
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))

# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.5))

# # Output layer
# model.add(Dense(1, activation='sigmoid'))

# # Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# history = model.fit(norm_X_train, y_train, epochs=100, batch_size=256, validation_split=0.1)

### Arsitektur 5

In [71]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

model = Sequential()

# Input layer
model.add(Dense(512, input_shape=(norm_X_train.shape[1],), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Hidden layers
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(norm_X_train, y_train, epochs=100, batch_size=256, validation_split=0.1)

Epoch 1/100
7/7 [==============================] - 2s 146ms/step - loss: 0.8300 - accuracy: 0.5863 - val_loss: 0.6680 - val_accuracy: 0.5419
Epoch 2/100
7/7 [==============================] - 1s 120ms/step - loss: 0.6553 - accuracy: 0.6683 - val_loss: 0.6458 - val_accuracy: 0.6536
Epoch 3/100
7/7 [==============================] - 1s 116ms/step - loss: 0.5684 - accuracy: 0.7373 - val_loss: 0.6261 - val_accuracy: 0.6983
Epoch 4/100
7/7 [==============================] - 1s 120ms/step - loss: 0.4463 - accuracy: 0.7988 - val_loss: 0.4841 - val_accuracy: 0.8492
Epoch 5/100
7/7 [==============================] - 1s 118ms/step - loss: 0.3039 - accuracy: 0.8807 - val_loss: 0.3666 - val_accuracy: 0.9553
Epoch 6/100
7/7 [==============================] - 1s 122ms/step - loss: 0.2196 - accuracy: 0.9137 - val_loss: 0.3086 - val_accuracy: 0.9330
Epoch 7/100
7/7 [==============================] - 1s 116ms/step - loss: 0.1648 - accuracy: 0.9422 - val_loss: 0.2422 - val_accuracy: 0.9609
Epoch 8/100
7

In [144]:
loss, accuracy = model.evaluate(norm_X_test, y_test)

7/7 [==============================] - 0s 6ms/step - loss: 0.3660 - accuracy: 0.9648


### Arsitektur 6

In [145]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU

model = Sequential()

# Input layer
model.add(Dense(1024, input_shape=(norm_X_train.shape[1],), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Hidden layers
model.add(Dense(512))
model.add(LeakyReLU(alpha=0.2))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(LeakyReLU(alpha=0.2))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU(alpha=0.2))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(LeakyReLU(alpha=0.2))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(norm_X_train, y_train, epochs=100, batch_size=256, validation_split=0.1)

Epoch 1/100
7/7 [==============================] - 4s 236ms/step - loss: 0.8297 - accuracy: 0.5957 - val_loss: 0.7522 - val_accuracy: 0.4078
Epoch 2/100
7/7 [==============================] - 2s 221ms/step - loss: 0.5733 - accuracy: 0.7366 - val_loss: 0.6515 - val_accuracy: 0.6536
Epoch 3/100
7/7 [==============================] - 2s 221ms/step - loss: 0.3770 - accuracy: 0.8348 - val_loss: 0.4744 - val_accuracy: 0.7933
Epoch 4/100
7/7 [==============================] - 2s 219ms/step - loss: 0.2330 - accuracy: 0.9112 - val_loss: 0.3538 - val_accuracy: 0.8827
Epoch 5/100
7/7 [==============================] - 2s 222ms/step - loss: 0.1311 - accuracy: 0.9571 - val_loss: 0.1813 - val_accuracy: 0.9721
Epoch 6/100
7/7 [==============================] - 2s 224ms/step - loss: 0.0810 - accuracy: 0.9770 - val_loss: 0.1347 - val_accuracy: 0.9721
Epoch 7/100
7/7 [==============================] - 2s 223ms/step - loss: 0.0487 - accuracy: 0.9894 - val_loss: 0.1127 - val_accuracy: 0.9777
Epoch 8/100
7

In [ ]:
history = model.fit(norm_X_train, y_train, epochs=100, batch_size=256, validation_split=0.1)

In [73]:
loss, accuracy = model.evaluate(norm_X_test, y_test)

7/7 [==============================] - 0s 12ms/step - loss: 0.1046 - accuracy: 0.9698


In [ ]:
model.save("my_model.h5")

### Model Jurnal Referensi

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
decision_tree = DecisionTreeClassifier(criterion='gini', splitter='best', min_samples_split=2,
                                       min_samples_leaf=1, max_features=None, random_state=42)
random_forest = RandomForestClassifier(n_estimators=100, criterion='gini',  random_state=42)
svm_model = SVC(C=1.0, kernel='poly', degree=1, coef0=1.0, gamma=1.0)
gradient_boosted_trees = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

In [ ]:
decision_tree.fit(norm_X_train, y_train)
random_forest.fit(norm_X_train, y_train)
svm_model.fit(norm_X_train, y_train)
gradient_boosted_trees.fit(norm_X_train, y_train)

In [ ]:
# Prediksi menggunakan data uji
decision_tree_pred = decision_tree.predict(norm_X_test)
random_forest_pred = random_forest.predict(norm_X_test)
svm_pred = svm_model.predict(norm_X_test)
gradient_boosted_trees_pred = gradient_boosted_trees.predict(norm_X_test)

In [ ]:
# Menghitung akurasi untuk masing-masing model
decision_tree_accuracy = accuracy_score(y_test, decision_tree_pred)
random_forest_accuracy = accuracy_score(y_test, random_forest_pred)
svm_accuracy = accuracy_score(y_test, svm_pred)
gradient_boosted_trees_accuracy = accuracy_score(y_test, gradient_boosted_trees_pred)

# Menampilkan hasil akurasi
print("Decision Tree Accuracy:", decision_tree_accuracy)
print("Random Forest Accuracy:", random_forest_accuracy)
print("SVM Accuracy:", svm_accuracy)
print("Gradient Boosted Trees Accuracy:", gradient_boosted_trees_accuracy)

### Model Testing

In [97]:
tfdf.columns

Index([               (0, nan),           (1, 'miquel'),
        (2, 'allowfullscreen'),         (3, 'guardian'),
                 (4, 'conced'),          (5, 'distort'),
                (6, 'replica'),    (7, '-year-minimum'),
               (8, 'strategi'),             (9, 'afgj'),
       ...
          (27074, 'westbrook'),        (27075, 'crane'),
           (27076, 'mosquera'),    (27077, 'well-paid'),
              (27078, 'magpi'), (27079, 'comedy-drama'),
           (27080, 'bamboozl'),        (27081, 'light'),
            (27082, 'fast-ag'),          (27083, 'auf')],
      dtype='object', length=27084)

In [74]:
def TFIDF(processed_unique_words,docs):
    tf = np.zeros((len(processed_unique_words), len(docs)))
    for i in range(len(docs)):
        for x in range(len(processed_unique_words)):
            count = docs[i].count(processed_unique_words[x])
            tf[x][i] = 0 if count == 0 else 1 + math.log(1 + count)
    return tf

In [116]:
# Contoh data teks (dokumen)
documents = [
    "COVID-19 Cure Found! Scientists have discovered a cure for COVID-19",
    "Breaking: UFO Sightings on the Rise! Multiple reports of UFO sightings around the world",
    "New Study: Chocolate is Healthy! Eating chocolate has been proven to improve health",
    "Government Denies Alien Contact Officials deny any contact with extraterrestrial beings",
    "Fake News Alert! Rumors of a zombie apocalypse are false, authorities confirm."
]

tf = TFIDF(tfdf.columns,documents)
tf = tf.transpose()

TypeError: must be str, not float

In [ ]:
predictions = model.predict(tf)
predicted_labels = [df['Label'].iloc[prediction.argmax()] for prediction in predictions]
print(predicted_labels)
for i in predicted_labels:
    if i == 1:
        print("True")
    else:
        print("Hoax bejir")